# 1 data reduction

In [5]:
#4.1 FRM提取行为变量
library(sqldf)
rfm_trad_flow<-read.csv("data/rfm_trad_flow.csv",fileEncoding="GBK")
rfm<-sqldf("select  cust_id,type,
           max(time) as Recency,count(*) as freq,sum(amount) as Monetary
           from  rfm_trad_flow
           where type='Special_offer' or type='Normal'
           group by cust_id,type")

head(rfm)

Loading required package: gsubfn
Loading required package: proto
Could not load tcltk.  Will use slower R code instead.
Loading required package: RSQLite
Loading required package: DBI


cust_id,type,Recency,freq,Monetary
10001,Normal,9/6/2010,15,3608
10001,Special_offer,10/12/2009,2,420
10002,Normal,7/29/2009,12,1894
10003,Normal,9/6/2009,15,3503
10003,Special_offer,1/2/2010,1,156
10004,Normal,9/13/2009,15,2979


In [6]:
#4.2 数据重组
#4.2.1 拆分列
rfm<-sqldf("select  cust_id,type,sum(amount) as Monetary
           from  rfm_trad_flow
           where type='Special_offer' or type='Normal'
           group by cust_id,type")

head(rfm)

cust_id,type,Monetary
10001,Normal,3608
10001,Special_offer,420
10002,Normal,1894
10003,Normal,3503
10003,Special_offer,156
10004,Normal,2979


In [7]:
library(reshape)
#https://www.r-statistics.com/tag/transpose/
rfm_w<- cast(rfm,cust_id~type)
rfm_w[is.na(rfm_w$Special_offer),]$Special_offer<-0 #进行缺失值替换
rfm_w$Special_offer_ratio<-rfm_w$Special_offer/(rfm_w$Special_offer+rfm_w$Normal)

head(rfm_w)

Using Monetary as value column.  Use the value argument to cast to override this choice


cust_id,Normal,Special_offer,Special_offer_ratio
10001,3608,420,0.10427011
10002,1894,0,0.00000000
10003,3503,156,0.04263460
10004,2979,373,0.11127685
10005,2368,0,0.00000000
10006,2534,58,0.02237654


In [8]:
#4.2.2 堆叠列
rfm_l<-melt(rfm_w,id="cust_id")

head(rfm_l)

,cust_id,value,type
Normal,10001,3608,Normal
Normal.1,10002,1894,Normal
Normal.2,10003,3503,Normal
Normal.3,10004,2979,Normal
Normal.4,10005,2368,Normal
Normal.5,10006,2534,Normal


In [9]:
#4.3 抽样
clients<-read.csv("data/clients.csv",fileEncoding="GBK")

In [10]:
#4.3.1简单随机抽样:随机选取100个客户用于调研
set.seed(100)
select<-sample(1:nrow(clients),100)
sample_clinet<-clients[select,]

head(sample_clinet)

,client_id,sex,birth_date,district_id
1653,1752,男,1978-11-23,70
1384,1462,男,1941-03-02,49
2965,3129,女,1946-08-01,5
303,322,男,1933-02-01,38
2514,2652,女,1949-05-09,41
2595,2742,女,1948-08-27,65


In [11]:
#简单随机抽样:随机选取10%个客户用于调研
set.seed(100)
select<-sample(1:nrow(clients),length(clients$client_id)*0.1)
sample_clinet<-clients[select,]
other_clinet<-clients[-select,]



In [12]:
#4.3.2分层抽样（STR）
library(sampling)
table(clients$district_id)
sample_clinet_stra=strata(clients,stratanames="district_id",
                          size=rep(5,times=77) ,method="srswor")

head(sample_clinet_stra)


  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20 
663  46  63  50  71  53  45  69  60  54  56  44  53  45  63  61  54  60  66  43 
 21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40 
 57  51  53  43  48  50  55  54  54  48  59  55  69  45  56  67  48  65  52  59 
 41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60 
 45  59  55  54  52  76  63  60  46  61  61  73  57 155  63  54  51  44  64  61 
 61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77 
 54  59  56 109  44  66  45  86  61 169  48 104  60 180  56  61  50 

,district_id,ID_unit,Prob,Stratum
2642,18,2642,0.083333333,1
3217,18,3217,0.083333333,1
3527,18,3527,0.083333333,1
3757,18,3757,0.083333333,1
4043,18,4043,0.083333333,1
3461,1,3461,0.007541478,2


##    4.2 分类变量的压缩

In [14]:
###数据集描述与属性说明###：
##数据说明：本数据是一份汽车贷款违约数据,因变量为是否违约(bad_ind)
##[T]因变量,[N]数值变量,[C]分类变量,[S]序数变量,[D]时间
##application_id	申请者ID
##account_number	帐户号
##[T]bad_ind	是否违约
##[D]vehicle_year	汽车购买时间
##[C]vehicle_make	汽车制造商
##[C]bankruptcy_ind	曾经破产标识
##[N]tot_derog	五年内信用不良事件数量(比如手机欠费消号)
##[N]tot_tr	全部帐户数量
##[N]age_oldest_tr	最久账号存续时间(月)
##[N]tot_open_tr	在使用帐户数量
##[N]tot_rev_tr	在使用可循环贷款帐户数量(比如信用卡)
##[N]tot_rev_debt	在使用可循环贷款帐户余额(比如信用卡欠款)
##[N]tot_rev_line	可循环贷款帐户限额(信用卡授权额度)
##[N]rev_util	可循环贷款帐户使用比例(余额/限额)
##[N]fico_score	FICO打分
##[N]purch_price	汽车购买金额(元)
##[N]msrp	建议售价
##[N]down_pyt	分期付款的首次交款
##[N]loan_term	贷款期限(月)
##[N]loan_amt	贷款金额
##[N]ltv	贷款金额/建议售价*100
##[N]tot_income	月均收入(元)
##[N]veh_mileage	行使历程(Mile)
##[C]used_ind	是否使用
##[N]weight	样本权重
##读取数据

accepts <- read.csv("data/accepts.csv",fileEncoding="GBK")

head(accepts)

application_id,account_number,bad_ind,vehicle_year,vehicle_make,bankruptcy_ind,tot_derog,tot_tr,age_oldest_tr,tot_open_tr,⋯,fico_score,purch_price,msrp,down_pyt,loan_term,loan_amt,ltv,tot_income,veh_mileage,used_ind
2314049,11613,1,1998,FORD,N,7,9,64,2,⋯,650,17200.00,17350,0.00,36,17200.00,99,6550.00,24000,1
63539,13449,0,2000,DAEWOO,N,0,21,240,11,⋯,649,19588.54,19788,683.54,60,19588.54,99,4666.67,22,0
7328510,14323,1,1998,PLYMOUTH,N,7,10,60,NA,⋯,613,13595.00,11450,0.00,60,10500.00,92,2000.00,19600,1
8725187,15359,1,1997,FORD,N,3,10,35,5,⋯,603,12999.00,12100,3099.00,60,10800.00,118,1500.00,10000,1
4275127,15812,0,2000,TOYOTA,N,0,10,104,2,⋯,764,26328.04,22024,0.00,60,26328.04,122,4144.00,14,0
8712513,16979,0,2000,DODGE,Y,2,15,136,4,⋯,680,26272.72,26375,0.00,36,26272.72,100,5400.00,1,0


##    4.4 分类变量的压缩

In [16]:
#4.4.1 分类变量重编码（概化）
#查看数据，确认不存在似不完整数据问题
table(accepts$vehicle_make,accepts$bad_ind)

               
                  0   1
                237  62
  3HYUNDAI        0   1
  ACCURA          2   0
  ACURA          10   1
  AUDI           16   1
  B50             1   0
  BMW            13   2
  BUICK          85  17
  BUIUCK          1   0
  CAD             1   1
  CADALLIC        1   0
  CADDY           1   0
  CADI            1   0
  CADILLAC       35   4
  CALERA          0   1
  CEHV            1   0
  CHCV            1   0
  CHERVOLET       1   0
  CHEV          137  33
  CHEVRELET       1   0
  CHEVROELT       1   0
  CHEVROLET     208  57
  CHEV X-CAB      1   0
  CHEVY         526 128
  CHEVY\\         0   1
  CHEVY GEO       1   1
  CHEVY S10       2   0
  CHEYV           0   2
  CHRY            6   4
  CHRYS           1   0
  CHRYSLER       79  15
  CHRYSLER JXI    1   0
  CHYRLER         0   1
  CHYRSLER        1   0
  CUTLASS         0   1
  DAEWOO          7   3
  DAEWOOD         1   0
  DODG            1   0
  DODGE         411 122
  DODGE RAM 4X4   1   0
